# Functional Interface

pytask offers a functional interface to users who want more flexibility than is given by a command line interface. It even allows you to run pytask from a Python interpreter or a Jupyter notebook like this article here.


Let's see how it works!

In [1]:
from pathlib import Path
from typing import Annotated

import pytask
from pytask import task

Here is a small workflow where two tasks create two text files and the third task merges both of them into one file.

One important bit to note here is that the second task is created from a lambda function. So, you can use dynamically defined functions to create tasks.

It also shows how easy it is to wrap any third-party function where you have no control over the signature, but you can still easily wrap them with pytask.

In [2]:
def task_create_first_file() -> Annotated[str, Path("first.txt")]:
    return "Hello, "


task_create_second_file = task(
    name="task_create_second_file", produces=Path("second.txt")
)(lambda *x: "World!")


def task_merge_files(
    first: Path = Path("first.txt"), second: Path = Path("second.txt")
) -> Annotated[str, Path("hello_world.txt")]:
    return first.read_text() + second.read_text()

Now, let us execute this little workflow.

In [3]:
session = pytask.build(
    tasks=[task_create_first_file, task_merge_files, task_create_second_file]
)

────────────────────────────────────────────── Start pytask session ───────────────────────────────────────────────

Platform: linux -- Python 3.11.5, pytask 0.4.0rc3.dev4+gfb5a25d.d20230930, pluggy 1.3.0

Root: /home/tobia/git/pytask

Collected 3 tasks.

Output()

╭─────────────────────────┬─────────╮
│ Task                    │ Outcome │
├─────────────────────────┼─────────┤
│ task_create_first_file  │ .       │
│ task_create_second_file │ .       │
│ task_merge_files        │ .       │
╰─────────────────────────┴─────────╯

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────── Summary ────────────╮
│  3  Collected tasks            │
│  3  Succeeded        (100.0%)  │
╰────────────────────────────────╯

──────────────────────────────────────────── Succeeded in 0.19 seconds ────────────────────────────────────────────

The information on the executed workflow can be found in the `session`.

In [4]:
session

Session(config={'pm': <pluggy._manager.PluginManager object at 0x7f3c1b438090>, 'markers': {'depends_on': 'Add dependencies to a task. See this tutorial for more information: [link https://bit.ly/3JlxylS]https://bit.ly/3JlxylS[/].', 'filterwarnings': 'Add a filter for a warning to a task.', 'persist': 'Prevent execution of a task if all products exist and even if something has changed (dependencies, source file, products). This decorator might be useful for expensive tasks where only the formatting of the file has changed. The state of the files which have changed will also be remembered and another run will skip the task with success.', 'produces': 'Add products to a task. See this tutorial for more information: [link https://bit.ly/3JlxylS]https://bit.ly/3JlxylS[/].', 'skip': 'Skip a task and all its dependent tasks.', 'skip_ancestor_failed': 'Internal decorator applied to tasks if any of its preceding tasks failed.', 'skip_unchanged': 'Internal decorator applied to tasks which have 

## Configuring the build

To configure the build, {func}`pytask.build` has many more options that are the same that you find on the commandline.

In [5]:
pytask.build?

Signature:
pytask.build(
    *,
    capture: "Literal['fd', 'no', 'sys', 'tee-sys'] | CaptureMethod" = <CaptureMethod.NO: 'no'>,
    check_casing_of_paths: 'bool' = True,
    config: 'Path | None' = None,
    database_url: 'str' = '',
    debug_pytask: 'bool' = False,
    disable_warnings: 'bool' = False,
    dry_run: 'bool' = False,
    editor_url_scheme: "Literal['no_link', 'file', 'vscode', 'pycharm'] | str" = 'file',
    expression: 'str' = '',
    force: 'bool' = False,
    ignore: 'Iterable[str]' = (),
    marker_expression: 'str' = '',
    max_failures: 'float' = inf,
    n_entries_in_table: 'int' = 15,
    paths: 'str | Path | Iterable[str | Path]' = (),
    pdb: 'bool' = False,
    pdb_cls: 'str' = '',
    s: 'bool' = False,
    show_capture: 'bool' = True,
    show_errors_immediately: 'bool' = False,
    show_locals: 'bool' = False,
    show_traceback: 'bool' = True,
    sort_table: 'bool' = True,
    stop_after_first_failure: 'bool' = False,
    strict_markers: 'bool' = Fals

In [6]:
# Cleanup
for name in ("first.txt", "second.txt", "hello_world.txt"):
    Path(name).unlink()